# Parsing Data from [datasport.com](https://www.datasport.com/en/)

We use postman to understand the parameters used by the url request, asked for the exercise.

(However, notice that there are equivalent tools for other browser - for instance, for firefox:
http://stackoverflow.com/questions/28997326/postman-addons-like-in-firefox)

In [14]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re

# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [2]:
form_source = rq.get("https://www.datasport.com/en/")
form_soup = bs4.BeautifulSoup(form_source.text, "html.parser")
# print(form_soup.prettify())

Let's get all the `select` menus of the page, using the `find_all` method of *BeautifulSoup* which allows to search for all tags of a certain type.

In [3]:
selectors = form_soup.find_all('select')
print(len(selectors))

4


Most importantly, we can find out what each tag is about by printing the its `name` attribute :

In [4]:
for num, s in enumerate(selectors):
    print("Select n°{} : {}".format(num, s.attrs['name'])) # wild french appears...

Select n°0 : etyp
Select n°1 : eventmonth
Select n°2 : eventyear
Select n°3 : eventlocation


In [5]:
for s in selectors:
    options = s.find_all('option')
    options_desc_values = [(o.text, o.attrs['value']) for o in options]
    print(s.attrs['name'] + ':')
    for (d,v) in options_desc_values:
        print("- {} [{}]".format(d,v)) # more french

etyp:
- All [all]
- ---- [all]
- Cross-Country-Skiing [Cross-Country-Skiing]
- Cycling [Cycling]
- Cycling,MTB [Cycling,MTB]
- Cycling,Others [Cycling,Others]
- Duathlon [Duathlon]
- Inline [Inline]
- MTB [MTB]
- MTB,Cycling [MTB,Cycling]
- MTB,Cycling,Others [MTB,Cycling,Others]
- MTB,Others [MTB,Others]
- MTB,X-Hours [MTB,X-Hours]
- Others [Others]
- Others,Inline,Running,MTB [Others,Inline,Running,MTB]
- Running [Running]
- Running,Inline [Running,Inline]
- Running,MTB [Running,MTB]
- Running,MTB,Others [Running,MTB,Others]
- Running,Skiing/Snowboard [Running,Skiing/Snowboard]
- Running,Waffenlauf [Running,Waffenlauf]
- Running,Walking [Running,Walking]
- Running,Walking,MTB [Running,Walking,MTB]
- Running,Walking,Others [Running,Walking,Others]
- Running,X-Hours [Running,X-Hours]
- Skiing/Snowboard [Skiing/Snowboard]
- Triathlon [Triathlon]
- Triathlon,Duathlon [Triathlon,Duathlon]
- Triathlon,Others [Triathlon,Others]
- Waffenlauf [Waffenlauf]
- Walking [Walking]
- X-Hours [X-Hour

## Let's get some data

In order to get started, we can now start collecting the results from the Lausanne marathone, one of the main early event in Switzerland.  

Understand the html of the main page, and __extract the relevant parameters__ to query:

In [6]:
laus_mar_url = 'https://services.datasport.com/2016/lauf/lamara/'
result_html = rq.get(laus_mar_url)

# use BS to get the classes in which the data is devided:

result_soup = bs4.BeautifulSoup(result_html.text, "lxml")
result_font = result_soup.find_all('font')

print('number of categories in the main page:', len(result_font))

number of categories in the main page: 119


In [7]:
# we look for the ones containing 
# '*** Overall ***', as they are the most general categories 

# this is indeed probably a GENERAL KEYWORD, as it's indeed found also in
# events in other laungauges, 
# like https://services.datasport.com/2016/lauf/ascona-locarno-marathon/

base_url = "https://services.datasport.com/2016/lauf/lamara"

good_fonts_num = []

links=[]
for n_font, font in enumerate(result_font):
#     print(font.findChild())
    if font.get('size')=='3':
        links_to_process=font.findAll('a')
        for link in links_to_process:
            link=str(link)
            try:
                link=link.split('"')[1]
                if link[:4]=='ALFA':
                    links.append(base_url+'/'+link)
            except:
                pass
        break

#     if 'Overall' in font.findChild().get_text():
            
#         good_fonts_num.append(n_font)
#         print(font.findChild().get_text())
        
        
# good_fonts_num = np.asarray(good_fonts_num)        
        
#  S***** -.- THERE IS A PROBLEM with the marathon hommes : 
# they are not in the same 'html shape' .. -.-

In [8]:
good_fonts_num

[]

Query the datasport.com with the right parameters and finally get the __tables__:

In [9]:
#base_url + '/' + rang_to_query[0]

In [57]:
final_list=[]
for link in links:
    # Get raw HTML response
    result_html = rq.get(link)#, params=rang_to_query[0])

    # Use BeautifulSoup and extract the first (and only) HTML table
    result_soup = bs4.BeautifulSoup(result_html.text, "lxml")
    # result_table = result_soup.find_all('table')[0]

    results=result_soup.findAll('font')
    del results[0]
    for table in results:
        if table.get('size')=='2':
            runner_list=str(table).split('¦')
            for runner in runner_list:
                runner=re.sub('<[^>]+>', ' ', runner)
                runner=re.sub('  +','&',runner)
                runner=runner.replace('\n','')
                runner=runner.replace(' \r','')
                if runner[0]==' ':
                    runner=runner[1:]
                start=runner[:3]
                runner=runner.split('&')
                if start=='10-' or start=='21-' or start=='42-':
                    final_list.append(runner)
    

    
    #df_trail = pd.read_html(result_soup.decode())

In [59]:
df = pd.DataFrame(final_list)
df.shape

(11248, 16)

In [60]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,21-H50,147.,Abaidia Jilani,1966 St-Légier-La Chiésaz,-----,"1:45.28,4","25.56,8",(5082),diplôme,foto,video,21-Hom,1241.,4.59,None,None
1,21-D40,81.,Abaidia Sandrine,1972 St-Légier,-----,"1:49.40,8","24.09,5",(5080),diplôme,foto,video,21-Fem,289.,5.11,None,None
2,21-H60,103.,Abb Jochen,1948 Ernen,-----,"2:50.40,7","1:21.28,7",(8381),diplôme,foto,video,21-Hom,2921.,8.05,None,None
3,10-H50,426.,Abbas Dhia,1961 Lausanne,-----,"1:13.04,1","38.13,0",(14163),diplôme,foto,video,10-Hom,2717.,7.18,None,None
4,21-H30,640.,Abbet Florian,1982 Pully,-----,"1:56.01,7","47.33,8",(3595),diplôme,foto,video,21-Hom,2012.,5.29,None,None
5,10-D30,517.,Abdala Maria Lucia,1979 Lausanne,-----,"1:01.30,8","27.09,1",(14054),diplôme,foto,video,10-Fem,1809.,6.09,None,None
6,10-H20,152.,Abdela Esa,1992 Pully,EVAM,-----,"42.44,1","14.26,0",(14299),diplôme,foto,video,10-Hom,517.,4.16,None
7,21-H20,67.,Abdelaziem Ahmed Ramy Bac,1992 Lausanne,-----,"1:29.06,1","20.32,9",(3116),diplôme,foto,video,21-Hom,230.,4.13,None,None
8,42-H30,38.,Abderhalden Matthias,1983 Chavannes-près-Renens,-----,"3:08.43,6","51.37,4",(367),diplôme,foto,video,42-Hom,85.,4.28,None,None
9,21-H60,3.,Abdesmed Mohamed,1956 F-Amiens,Amicale Val de somme,-----,"1:33.24,8","4.12,8",(2540),diplôme,foto,video,21-Hom,411.,4.25,None


# ******* ******* ******* ******* *******  
# OLD CODE 
# ******* ******* ******* ******* ******* ******* 

In [ ]:
df = pd.read_html(result_table.decode())[0]
df.head()

In [ ]:
df.columns = df.loc[1]                # use row 2 as column names
df = df.drop([0, 1])                  # drop useless first rows
df = df.drop([np.nan], axis=1)        # drop useless nan column
df.index = df['No Sciper']            # use sciper column as index

# Drop some columns
df = df.drop(['Orientation Bachelor', 'Orientation Master', 'Filière opt.', 'Type Echange', 'Ecole Echange'], axis=1)

# Do some renaming
df.index.name = 'sciper'
df.columns = ['gender', 'full_name', 'specialization', 'minor', 'status', 'sciper']

# Map gender to more standard names
dict_gender = {'Monsieur': 'male','Madame': 'female'}
df.gender.replace(dict_gender, inplace=True)
df.head()

## Some tools

We can define a helper function which, given a base URL and a dictionary of parameters, will fetch the data and fill a DataFrame with it.

In [ ]:
def get_data(base_url, params_dict):
    """Get data from IS-Academia in a pandas DataFrame"""
    
    # Same sequence of operations of above, with a check if the result_table is empty
    
    result_html = rq.get(base_url,params=params_dict)
    result_soup = bs4.BeautifulSoup(result_html.text, "lxml")
    result_table = result_soup.find_all('table')[0]
    
    if (result_table.text == ''):
        # Return empty dataframe
        df = pd.DataFrame()
    else:
        # Build a DataFrame containing the data, with SCIPER as index
        df = pd.read_html(result_table.decode())[0]
        try:
            df.columns = df.loc[1]                # use 2nd row as column names
            df = df.drop([0, 1])                  # drop useless first rows
            df = df.drop([np.nan], axis=1)        # drop useless nan column
            df.index = df['No Sciper']            # use sciper column as index
        
            # Drop some columns
            df = df.drop(['Orientation Bachelor', 'Orientation Master', 'Filière opt.', 'Type Echange', 'Ecole Echange'], axis=1)
            # Do some renaming
            df.index.name = 'sciper'
            df.columns = ['gender', 'full_name', 'specialization', 'minor', 'status', 'sciper']
            # Map gender to more standard names
            dict_gender = {'Monsieur': 'male','Madame': 'female'}
            df.gender.replace(dict_gender, inplace=True)
        except:
            df = pd.DataFrame()
    
    return df

The following lines test this function with hardcoded values :

In [ ]:
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?"
params_dict = {
    'ww_x_GPS': 2021043255,
    'ww_i_reportModel': 133685247,
    'ww_i_reportModelXsl': 133685270,
    'ww_x_UNITE_ACAD': 249847,
    'ww_x_PERIODE_ACAD': 355925344,
    'ww_x_PERIODE_PEDAGO': 249108,
    'ww_x_HIVERETE':2936286
}

get_data(base_url, params_dict).head()

Finally let's get all the possible values in a cleaner way and keep them in variables that we will use throughout this notebook.

In [ ]:
acad_period = {}
level = {}
semester = {}
acad_unit = {}

for s in selectors:
    options = s.find_all('option')
    options_desc_values = [(o.text, o.attrs['value']) for o in options]
    s_name = s.attrs['name']
    choices = {d: int(v) for (d,v) in options_desc_values if d!=''}
    
    if s_name == 'ww_x_PERIODE_ACAD':
        acad_period = choices
    elif s_name == 'ww_x_PERIODE_PEDAGO':
        level = choices
    elif s_name == 'ww_x_HIVERETE':
        for (d,v) in options_desc_values:
            if 'automne' in d:
                semester['automne'] = int(v)
            elif 'printemps' in d:
                semester['printemps'] =int(v)
    elif s_name == 'ww_x_UNITE_ACAD':
        acad_unit = choices

# Example of result
acad_period

### Store data locally

In [ ]:
# Get bachelor data for every year and store it if it's not empty
import os
local_dir = '.local-data'
try:
    os.mkdir(local_dir)
except FileExistsError:
    # directory exists
    print("Using existing '" + local_dir + "' directory")

In [ ]:
# Fixed values
params_dict = {
    'ww_x_GPS': -1,
    'ww_i_reportModel': 133685247,
    'ww_i_reportModelXsl': 133685270,
    'ww_x_UNITE_ACAD': acad_unit['Informatique']
}

# Iterate over all the varying params and keep only data for bachelors
for year_key, year_value in acad_period.items():
    for level_key, level_value in level.items():
        for semester_key, semester_value in semester.items():
            if 'bachelor' in level_key.lower():
                params_dict['ww_x_PERIODE_ACAD'] = year_value
                params_dict['ww_x_PERIODE_PEDAGO'] = level_value
                params_dict['ww_x_HIVERETE'] = semester_value
                
                df = get_data(base_url, params_dict)
                if not df.empty:
                    # Persist dataframe locally with pickle
                    filename = year_key + '-' + level_key.replace(' ', '-').lower() + '-' + semester_key
                    df.to_pickle(local_dir + '/' + filename)

In [ ]:
# the previous cell should download 60 files!, as you can check with this command:
print(len([name for name in os.listdir(local_dir)]))

We hereby show an example of dataframe laoded from the files previously download:

In [ ]:
df_example = pd.read_pickle(local_dir + '/2007-2008-bachelor-semestre-6-printemps')
df_example.head()